# Prompt stretegies

In [57]:
import os
from dotenv import load_dotenv
from langchain_community.graphs import Neo4jGraph

In [58]:
db = Neo4jGraph(url = os.environ["NEO4J_URI"],
                 username=os.environ["NEO4J_USERNAME"],
                 password=os.environ["NEO4J_PASSWORD"],
                 refresh_schema=True )

In [59]:
moview_query="""
LOAD CSV WITH HEADERS FROM
'https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/movies/movies_small.csv' as row

MERGE(m:Movie{id:row.movieId})
SET m.released = date(row.released),
    m.title = row.title,
    m.imdbRating = toFloat(row.imdbRating)
FOREACH (director in split(row.director, '|') | 
    MERGE (p:Person {name:trim(director)})
    MERGE (p)-[:DIRECTED]->(m))
FOREACH (actor in split(row.actors, '|') | 
    MERGE (p:Person {name:trim(actor)})
    MERGE (p)-[:ACTED_IN]->(m))
FOREACH (genre in split(row.genres, '|') | 
    MERGE (g:Genre {name:trim(genre)})
    MERGE (m)-[:IN_GENRE]->(g))


"""
db.query(moview_query)
print(db.schema)

Node properties:
Territory {territoryID: STRING, territoryDescription: STRING}
Region {regionID: STRING, regionDescription: STRING}
Supplier {supplierID: STRING, companyName: STRING, contactName: STRING, contactTitle: STRING, address: STRING, city: STRING, region: STRING, postalCode: STRING, country: STRING, phone: STRING, fax: STRING, homePage: STRING}
Shipper {companyName: STRING, phone: STRING, shipperID: STRING}
Product {productID: STRING, productName: STRING, unitPrice: FLOAT, unitsInStock: INTEGER, unitsOnOrder: INTEGER, reorderLevel: INTEGER, discontinued: BOOLEAN}
Category {categoryID: STRING, categoryName: STRING, description: STRING}
Order {orderID: STRING, orderDate: STRING, requiredDate: STRING, shippedDate: STRING, freight: STRING, shipName: STRING, shipAddress: STRING, shipCity: STRING, shipRegion: STRING, shipPostalCode: STRING, shipCountry: STRING}
Customer {companyName: STRING, contactName: STRING, contactTitle: STRING, address: STRING, city: STRING, region: STRING, po

In [60]:
groq_api_key = os.getenv('GROQ_API_KEY')
from langchain_groq import ChatGroq
llm = ChatGroq(model="Gemma2-9b-it",groq_api_key= groq_api_key)
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7faa41842d10>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7faa41843850>, model_name='Gemma2-9b-it')

In [61]:
from langchain.chains import GraphCypherQAChain
chain = GraphCypherQAChain.from_llm(graph= db,llm=llm,verbose=True)
chain

GraphCypherQAChain(verbose=True, graph=<langchain_community.graphs.neo4j_graph.Neo4jGraph object at 0x7faa4216c640>, cypher_generation_chain=LLMChain(prompt=PromptTemplate(input_variables=['question', 'schema'], template='Task:Generate Cypher statement to query a graph database.\nInstructions:\nUse only the provided relationship types and properties in the schema.\nDo not use any other relationship types or properties that are not provided.\nSchema:\n{schema}\nNote: Do not include any explanations or apologies in your responses.\nDo not respond to any questions that might ask anything else than for you to construct a Cypher statement.\nDo not include any text except the generated Cypher statement.\n\nThe question is:\n{question}'), llm=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7faa41842d10>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7faa41843850>, model_name='Gemma2-9b-it')), qa_chain=LLMChain(prompt=PromptTemplate(input_v

In [62]:
chain.graph_schema

'Node properties are the following:\nTerritory {territoryID: STRING, territoryDescription: STRING},Region {regionID: STRING, regionDescription: STRING},Supplier {supplierID: STRING, companyName: STRING, contactName: STRING, contactTitle: STRING, address: STRING, city: STRING, region: STRING, postalCode: STRING, country: STRING, phone: STRING, fax: STRING, homePage: STRING},Shipper {companyName: STRING, phone: STRING, shipperID: STRING},Product {productID: STRING, productName: STRING, unitPrice: FLOAT, unitsInStock: INTEGER, unitsOnOrder: INTEGER, reorderLevel: INTEGER, discontinued: BOOLEAN},Category {categoryID: STRING, categoryName: STRING, description: STRING},Order {orderID: STRING, orderDate: STRING, requiredDate: STRING, shippedDate: STRING, freight: STRING, shipName: STRING, shipAddress: STRING, shipCity: STRING, shipRegion: STRING, shipPostalCode: STRING, shipCountry: STRING},Customer {companyName: STRING, contactName: STRING, contactTitle: STRING, address: STRING, city: STRING

In [63]:
examples = [
    {
        "question": "How many artists are there?",
        "query": "MATCH (a:Person)-[:ACTED_IN]->(:Movie) RETURN count(DISTINCT a)",
    },
    {
        "question": "How many movies has Tom Hanks acted in?",
        "query": "MATCH (a:Person {name: 'Tom Hanks'})-[:ACTED_IN]->(m:Movie) RETURN count(m)",
    },
    {
        "question": "List all the genres of the movie Schindler's List",
        "query": "MATCH (m:Movie {{title: 'Schindler\\'s List'}})-[:IN_GENRE]->(g:Genre) RETURN g.name",
    },
    {
        "question": "Which actors have worked in movies from both the comedy and action genres?",
        "query": "MATCH (a:Person)-[:ACTED_IN]->(:Movie)-[:IN_GENRE]->(g1:Genre), (a)-[:ACTED_IN]->(:Movie)-[:IN_GENRE]->(g2:Genre) WHERE g1.name = 'Comedy' AND g2.name = 'Action' RETURN DISTINCT a.name",
    },
    {
        "question": "Which directors have made movies with at least three different actors named 'John'?",
        "query": "MATCH (d:Person)-[:DIRECTED]->(m:Movie)<-[:ACTED_IN]-(a:Person) WHERE a.name STARTS WITH 'John' WITH d, COUNT(DISTINCT a) AS JohnsCount WHERE JohnsCount >= 3 RETURN d.name",
    },
    {
        "question": "Identify movies where directors also played a role in the film.",
        "query": "MATCH (p:Person)-[:DIRECTED]->(m:Movie), (p)-[:ACTED_IN]->(m) RETURN m.title, p.name",
    },
    {
        "question": "Find the actor with the highest number of movies in the database.",
        "query": "MATCH (a:Actor)-[:ACTED_IN]->(m:Movie) RETURN a.name, COUNT(m) AS movieCount ORDER BY movieCount DESC LIMIT 1",
    },
]

In [65]:
from langchain_core.prompts import FewShotPromptTemplate,PromptTemplate

example_prompt=PromptTemplate.from_template(
    "User input:{question}\n Cypher query:{query}"
)

prompt=FewShotPromptTemplate(
    examples=examples[:5],
    example_prompt=example_prompt,
    prefix="You are a Neo4j expert. Given an input question,create a syntactically very accurate Cypher query",
    suffix="User input: {question}\nCypher query: ",
    input_variables=["question","schema"]
)

In [66]:
prompt

FewShotPromptTemplate(input_variables=['question'], examples=[{'question': 'How many artists are there?', 'query': 'MATCH (a:Person)-[:ACTED_IN]->(:Movie) RETURN count(DISTINCT a)'}, {'question': 'How many movies has Tom Hanks acted in?', 'query': "MATCH (a:Person {name: 'Tom Hanks'})-[:ACTED_IN]->(m:Movie) RETURN count(m)"}, {'question': "List all the genres of the movie Schindler's List", 'query': "MATCH (m:Movie {{title: 'Schindler\\'s List'}})-[:IN_GENRE]->(g:Genre) RETURN g.name"}, {'question': 'Which actors have worked in movies from both the comedy and action genres?', 'query': "MATCH (a:Person)-[:ACTED_IN]->(:Movie)-[:IN_GENRE]->(g1:Genre), (a)-[:ACTED_IN]->(:Movie)-[:IN_GENRE]->(g2:Genre) WHERE g1.name = 'Comedy' AND g2.name = 'Action' RETURN DISTINCT a.name"}, {'question': "Which directors have made movies with at least three different actors named 'John'?", 'query': "MATCH (d:Person)-[:DIRECTED]->(m:Movie)<-[:ACTED_IN]-(a:Person) WHERE a.name STARTS WITH 'John' WITH d, COUNT

In [67]:
llm
chain=GraphCypherQAChain.from_llm(graph=db,llm=llm,cypher_prompt=prompt,verbose=True)

In [70]:
print(prompt.format(question="How many artists are there?",schema="foo"))


KeyError: 'name'